In [ ]:
import os
import struct
import numpy as np
import pandas as pd

### reference
   - [stackoverflow](https://stackoverflow.com/questions/16444726/binary-representation-of-float-in-python-bits-not-hex)
   - [struct 라이브러리](https://docs.python.org/3/library/struct.html#format-strings)

In [ ]:
def binary(num,big_endian=True, bit_16=True):
    # big_endian : True 이면, big_endian format으로 출력, 아니면 little_endian format으로 출력
    # bit_16 : True 이면, 16bit float으로 출력, 아니면 32bit float으로 출력
    endian_format = ">" if big_endian else '<'
    bit_format = "e" if bit_16 else 'f'
    num_format = endian_format + bit_format
    struct.pack(num_format, num)
    return ''.join(bin(c).replace('0b', '').rjust(8, '0') for c in struct.pack(num_format, num)[::-1])

| float 16| sign(1) | expo(5) | remain(10) |
|---------|---------|---------|------------|
| float 32| sign(1) | expo(8) | remain(23) |

### tb_convert에 이용되는 테스트 데이터셋

In [ ]:
# 16bit - Big Endian value list
with open("../input/tb_convert_16be.txt",'w') as f:
    for i in range(-10,10):
        value = (1+2**(-3)+2**(-2)) * 2**(i)
        line = (binary(value,big_endian=True,bit_16=True))+'0'*16 + '\n'
        f.write(line)
        
# 16bit - Big Endian & 32bit - little Endian
df = pd.DataFrame(columns=['value','16bit-BigEndian','32bit-LittleEndian'])
for i in range(-10,10):
    value = (1+2**(-3)+2**(-2)) * 2**(i)
    be_16 = (binary(value,big_endian=True,bit_16=True))+'0'*16
    le_32 = binary(value,big_endian=False,bit_16=False)
    df = df.append({'value':value, "16bit-BigEndian":be_16,"32bit-LittleEndian":le_32},ignore_index=True)
df.to_csv("../answer/tb_convert_16be_answer.csv",index=False,float_format="%03.13f")

### tb_basic_pe에 이용되는 테스트 데이터셋

### 0. tb_basic_pe_ex0.txt
    => 수업시간에 준 input.txt

### 1. tb_basic_pe_ex1.txt

* value type : 32bit little-endian

* vector_size : 64

* 출력 형태 : 
2개의 동일 길이 벡터를 곱함

In [ ]:
def hex_to_float(h):
    return struct.unpack('>f',bytes.fromhex(h))[0]

In [ ]:
def float_to_hex(f):
    return hex(struct.unpack('>I', struct.pack('>f', f))[0]).replace('0x','')

In [ ]:
first_vector = np.linspace(0.5,2.5,num=64)
second_vector = np.linspace(1,3,num=64)

In [ ]:
result = np.sum(first_vector*second_vector)
print("return value:",result,"\nhex value : ",float_to_hex(result))

In [ ]:
with open("../input/tb_basic_pe_ex1.txt",'w') as file:
    for f in first_vector:
        file.write(float_to_hex(f)+'\n')
    for f in second_vector:
        file.write(float_to_hex(f)+'\n')